In [1]:
"""ATE: Automatic Term Extraction
"""

import pandas as pd

import ginza # pip install -U ginza ja_ginza
import nltk  # pip install nltk
import spacy # pip install nltk
import pke   # pip install git+https://github.com/boudinfl/pke.git

In [2]:
file_path = "datasets/生体医工学会論文5755件.csv"
df = pd.read_csv(file_path,encoding='utf-8')

In [33]:
df[df['Published_Year']==2018][['Title','Published_Year','Vol', 'No']].head(10)

,Title,Published_Year,Vol,No
2777,電気インピーダンスの周波数特性計測による細胞の生存判別,2018,Annual56,Abstract
2778,対極板の違いにより発生する低周波雑音が術中モニタリングに与える影響,2018,Annual56,Abstract
2779,センチネルリンパ節内の磁気ナノ粒子定量化装置の開発,2018,Annual56,Abstract
2780,微小試料の画像計測のためのMRI観測システム,2018,Annual56,Abstract
2781,痛風発作を誘発する尿酸ナトリウム結晶の磁気と光による検出,2018,Annual56,Abstract
2782,PPGとドップラーレーダを用いた収縮期血圧のカフレス連続測定,2018,Annual56,Abstract
2783,励起と検出が異なる種類の信号伝送により行われるパッシブテレメトリ,2018,Annual56,Abstract
2784,EVAHEART２承認、チップレスカニューラ薬事申請、ならびに中国・米国治験の展開について,2018,Annual56,Abstract
2785,ヒト体表のバイオフォトン画像及び分光分析による酸化ストレス計測法の検討,2018,Annual56,Abstract
2786,電気化学発光を利用した生体貼付型酸素センサの開発,2018,Annual56,Abstract


In [4]:
years = df["Published_Year"].unique()
years

array([2005, 2006, 2007, 2008, 2009, 2010, 2011, 2012, 2013, 2014, 2015,
       2016, 2017, 2018, 2019, 2020, 2021, 2022, 2023])

In [5]:
docs = {}
for y in years:
    docs[y] = df[(df['Published_Year']==y)]['Title'].values.tolist()
    
titles = docs[2022]
print("\n".join(titles[:10]))

生体医工学ウェブ辞典（第一分冊）
市販・試作ウエアラブルデバイスのフィールド適用事例の紹介と課題抽出
飲料摂取をともなうデスクワーク中の生理反応から見た遠隔生体計測の課題
ウエアラブルデバイスによる心拍と呼吸データの比較事例
日常・産業場面におけるウエアラブルデバイス利活用のための問題意識
NIRSを用いた認知介入による認知機能の改善効果研究の最前線
近赤外分光法を用いた脳機能計測アプリケーションの展開
近赤外分光計測を用いたハイパースキャンデータの解析
機能性近赤外分光装置のためのソフトウェア開発の現状と将来展望
拡散相関分光法による組織血流イメージング：原理と応用


In [6]:
def get_key_phrase(spacy_model, text, n = 5, stopwords=None):
    """キーフレーズ処理

    Args:
        text ([type]): キーフレーズ対象の文字列
        n (int, optional): [description]. Defaults to 10.
    Returns:
        [type]: [description]
    """
    #MultipartiteRankなどのキーフレーズの手法については以下のサイトが参考になります
    #https://www.ogis-ri.co.jp/otc/hiroba/technical/similar-document-search/part5.html
    extractor = pke.unsupervised.MultipartiteRank()
    extractor.load_document(input=text, language='ja', normalization=None, stoplist=stopwords)
    extractor.candidate_selection( pos={'NOUN', 'PROPN', 'ADJ', 'NUM'})
    extractor.candidate_weighting(threshold=0.10, method='average', alpha=1.1)
    key_phrase = extractor.get_n_best(n)
    
    return  key_phrase

In [7]:
from spacy.lang.ja import stop_words
pke.lang.stopwords['ja_ginza'] = 'ja'
spacy_model = spacy.load("ja_ginza")  # GiNZAモデルの読み込み
stopwords = list(stop_words.STOP_WORDS)
stopwords.extend(['検討','研究','開発','展望','事例','課題','試作','提案']) 
print(stopwords)
nltk.corpus.stopwords.words = stopwords

['て', 'なく', 'ながら', 'いう', 'へ', 'さらに', 'つつ', 'だ', 'まま', 'いずれ', 'でき', 'なる', 'すべて', 'もの', 'ほとんど', 'み', 'よく', 'あまり', 'ず', 'いる', 'それぞれ', 'せ', 'なお', 'とき', 'つ', 'なし', 'にて', 'すぐ', 'き', 'よう', 'ち', 'もと', 'きっかけ', 'いく', 'おら', 'ある', 'もう', 'し', 'なら', 'よっ', 'せい', 'のみ', 'ほか', 'だけ', 'え', 'ため', 'つけ', 'おり', 'ここ', 'が', 'くん', 'そこ', 'る', 'こう', 'こ', 'よ', 'あれ', 'さ', 'ところ', 'さん', 'あっ', 'かつ', 'と', 'ちゃん', 'など', 'のち', 'で', 'より', 'な', 'ほぼ', 'どう', 'ん', 'ご', 'ない', 'くる', 'なっ', 'たい', 'これ', 'いっ', 'た', 'を', 'ただし', 'そして', '一', 'から', 'せる', 'ら', 'れ', 'あ', 'の', 'しか', 'および', 'つい', 'ぶり', 'たり', 'なり', 'べき', 'ひと', 'ぬ', 'お', 'こと', 'おい', 'する', 'また', 'しまう', 'しかし', 'なけれ', 'まで', 'よれ', 'いつ', 'かつて', 'たち', 'だっ', 'ま', 'い', 'らしい', 'そう', 'しまっ', 'その', 'やっ', 'よる', 'とも', 'に', 'なかっ', 'も', 'はじめ', 'られ', 'しよう', 'れる', 'あり', 'たら', 'できる', 'うち', 'す', 'られる', 'それ', 'や', 'あるいは', 'おけ', 'かなり', 'か', 'ごと', 'は', 'もっ', 'かけ', 'ほど', 'この', 'とっ', 'ば', 'ます', 'ね', 'いわ', 'いい', 'です', '検討', '研究', '開発', '展望', '事例', '課題', '試作', '提案']


In [8]:
text = "。\n".join(docs[2022][:10])
print(text)
get_key_phrase(spacy_model, text, n=150, stopwords=stopwords)

生体医工学ウェブ辞典（第一分冊）。
市販・試作ウエアラブルデバイスのフィールド適用事例の紹介と課題抽出。
飲料摂取をともなうデスクワーク中の生理反応から見た遠隔生体計測の課題。
ウエアラブルデバイスによる心拍と呼吸データの比較事例。
日常・産業場面におけるウエアラブルデバイス利活用のための問題意識。
NIRSを用いた認知介入による認知機能の改善効果研究の最前線。
近赤外分光法を用いた脳機能計測アプリケーションの展開。
近赤外分光計測を用いたハイパースキャンデータの解析。
機能性近赤外分光装置のためのソフトウェア開発の現状と将来展望。
拡散相関分光法による組織血流イメージング：原理と応用


[('ウエアラブル デバイス 利活用', 0.12535370369887078),
 ('認知 介入', 0.12195927604415156),
 ('認知 機能', 0.11989406339398705),
 ('問題 意識', 0.11432452298690556),
 ('産業 場面', 0.10740202803665067),
 ('遠隔 生体 計測', 0.08765361070320592),
 ('呼吸 データ', 0.08211644637787247),
 ('最前線', 0.08169748487306891),
 ('生理 反応', 0.07847261021733803),
 ('ハイパー スキャン データ', 0.046598353249644384),
 ('組織 血流 イメージング', 0.034527900418305)]

In [20]:
y = 2019
text = "。\n".join(docs[y][:500])
print(len(docs[y]))
get_key_phrase(spacy_model, text, n=100, stopwords=stopwords)

607


[('可能性', 0.007333070924271722),
 ('取り組み', 0.006098000393862529),
 ('心拍 変動 解析', 0.003908630038821869),
 ('人工知能', 0.0037682696526809565),
 ('再生 医療', 0.0037198226056769403),
 ('可視化', 0.003713607344104096),
 ('医療 応用', 0.003700169385875534),
 ('特徴 抽出', 0.0036574858112964305),
 ('電波 利用', 0.003630792992554362),
 ('ノイズ', 0.003559675398760222),
 ('レギュラトリーサイエンス', 0.003266409917951535),
 ('トイレ', 0.003114596690444751),
 ('パラメータ', 0.003063134788791899),
 ('頭皮 血流', 0.0030413323363730925),
 ('看護 応用', 0.003006549409787104),
 ('妥当性', 0.002940149309648655),
 ('標準化', 0.00291202512908061),
 ('安全性', 0.0028654848788348362),
 ('スマートフォン', 0.0028597567637425264),
 ('関係性', 0.002806515182759968),
 ('呼吸 位相 計測', 0.002805990832848187),
 ('生理学', 0.002756354856796342),
 ('業務 改善', 0.0027495052526400458),
 ('相関性', 0.002735711012309391),
 ('実用化', 0.002600809296649505),
 ('動作 解析', 0.002579900822652922),
 ('発生 メカニズム', 0.002519808995673538),
 ('評価方法', 0.0024836465148533847),
 ('カフレス 血圧 推定', 0.0024486272691751935),
 ('医療機器'

In [35]:
y = 2020
text = "。\n".join(docs[y][:500])
print(len(docs[y]))
terms = get_key_phrase(spacy_model, text, n=100, stopwords=stopwords)
for word, v in terms:
    print(f"{word} & ${v:.6f}$ \\\\ ")

445
可視化 & $0.013381$ \\ 
可能性 & $0.007333$ \\ 
有用性 & $0.006709$ \\ 
映像 脈波 & $0.005941$ \\ 
末梢 血行 動態 & $0.005366$ \\ 
精度 検証 & $0.005245$ \\ 
モニタリング & $0.005038$ \\ 
心電図 & $0.004960$ \\ 
定量 評価 & $0.004485$ \\ 
血行 動態 計測 & $0.004070$ \\ 
血圧計 & $0.004037$ \\ 
日本人 & $0.003969$ \\ 
生体 リズム & $0.003944$ \\ 
関係性 & $0.003897$ \\ 
上行 大動脈 & $0.003816$ \\ 
推定 方法 & $0.003659$ \\ 
ダブル ルーメン カテーテル & $0.003632$ \\ 
ヒト ips細胞 由来 心筋 細胞 & $0.003591$ \\ 
自転車 クランク & $0.003585$ \\ 
細胞 外電 & $0.003574$ \\ 
最適 設計 & $0.003573$ \\ 
ウェアラブル 心電図 計測 技術 & $0.003531$ \\ 
容量 結合 方式 & $0.003498$ \\ 
最適化 & $0.003485$ \\ 
多元 計算 解剖 モデル & $0.003474$ \\ 
空間的 配置 条件 & $0.003470$ \\ 
走査 方法 & $0.003444$ \\ 
超音波 音源 & $0.003424$ \\ 
健常者 & $0.003416$ \\ 
超音波 プローブ 動作 補助 機器 & $0.003397$ \\ 
最適 先端 形状 & $0.003326$ \\ 
取り組み & $0.003318$ \\ 
同時 計測 & $0.003252$ \\ 
エアトラップチャンバ & $0.003226$ \\ 
キャビテーション & $0.003218$ \\ 
理想 形状 & $0.003216$ \\ 
キャリアパス & $0.003180$ \\ 
ワークライフバランス & $0.003159$ \\ 
マウス & $0.003137$ \\ 
拍動 循環 シミュレータ & $0.003133$ \\ 
非接

In [21]:
y = 2021
text = "。\n".join(docs[y][:500])
print(len(docs[y]))
get_key_phrase(spacy_model, text, n=100, stopwords=stopwords)

583


[('性能 評価', 0.007853778758721688),
 ('心拍 変動', 0.00699240002118784),
 ('可視化', 0.006004138250579396),
 ('極細 カテーテル', 0.005918664759373996),
 ('有用性', 0.005448741918071895),
 ('高齢者', 0.005317055048980414),
 ('構造 解析', 0.005033270820969611),
 ('血液 凝固', 0.005017180187487929),
 ('筋電図', 0.0049981506767465995),
 ('脱血 不良', 0.004904149569682899),
 ('擬似 クレンチング', 0.00405816507032222),
 ('化学 応答', 0.003986011037843109),
 ('関連性', 0.003893841512266293),
 ('シミュレーション', 0.00380420699740351),
 ('睡眠 段階', 0.003743651660572288),
 ('血管 組織', 0.003707072850328691),
 ('ブラキシズム', 0.003693688957752658),
 ('ノイズ', 0.003651123552490932),
 ('日常 会話', 0.003618379909289418),
 ('メカニズム', 0.0036014086827566073),
 ('mci スクリーニング', 0.003529268362245049),
 ('薬剤 反応 再現', 0.003486772612147335),
 ('コラーゲンゲルチューブ', 0.003472952661776407),
 ('神経 活動', 0.0034589415750579384),
 ('自動 検出', 0.00345852394173496),
 ('dlc', 0.0034543989223155846),
 ('mr 画像', 0.0034103116715913893),
 ('自動 認識', 0.0033807977870785886),
 ('ネガティビティ', 0.003365733744013699)

In [22]:
y = 2022
text = "。\n".join(docs[y][:500])
print(len(docs[y]))
get_key_phrase(spacy_model, text, n=100, stopwords=stopwords)

391


[('可視化', 0.010763155673175878),
 ('lcc 方式 補償 回路', 0.006921229981493105),
 ('妥当性', 0.005711715579732388),
 ('可能性', 0.005329916085088529),
 ('骨伝導', 0.005251555733964972),
 ('認知 機能', 0.004886273964744659),
 ('身体 活動量', 0.004554505944518493),
 ('最適 設計', 0.004336932174499975),
 ('シミュレーション', 0.004018835955397273),
 ('米国 糖尿病 患者', 0.0040179063984160665),
 ('活動 パターン', 0.003962963237621026),
 ('臨床 応用', 0.003914189072984731),
 ('モニタリングシステム', 0.0038572941205917335),
 ('周波数 400 khz', 0.003845109910528897),
 ('送電 コイル', 0.003752599453152514),
 ('顔面 部位', 0.0037382520520903334),
 ('血圧 推定', 0.003707031760598887),
 ('面積 推定', 0.003685112437222413),
 ('pet', 0.0036441928509483),
 ('性能 向上', 0.0036423961823598265),
 ('fdg', 0.0036143945578031383),
 ('認知 機能 低下', 0.0036075835549578424),
 ('成人 男性', 0.0036050190932245514),
 ('マーカ 領域', 0.0035859885834323402),
 ('血管 抵抗 指数', 0.003561548875868793),
 ('同時 計測', 0.0035348273221304733),
 ('超音波 画像', 0.0035086218097175716),
 ('抑うつ 状態 評価', 0.0034557012028727445),
 ('把握力 調整 

In [23]:
y = 2023
text = "。\n".join(docs[y][:500])
print(len(docs[y]))
get_key_phrase(spacy_model, text, n=100, stopwords=stopwords)

494


[('可視化', 0.013302776431301387),
 ('定量 評価', 0.006715983935948926),
 ('有用性', 0.005344349999176621),
 ('シミュレーション', 0.00518207154960376),
 ('net', 0.004751030846867686),
 ('pet', 0.003598496705059983),
 ('事象 関連 電位', 0.003194128531582414),
 ('領域 分割', 0.003187651463185213),
 ('rma', 0.003093076107857001),
 ('fdg', 0.0030817910889768506),
 ('可能性', 0.003066220006810968),
 ('cfd 解析', 0.003029324916282338),
 ('研究開発', 0.0030262240925103805),
 ('カメラ', 0.003000841594170302),
 ('超音波', 0.002999235776873844),
 ('心拍 変動', 0.002965826351546002),
 ('自動 抽出', 0.002961450649373811),
 ('感情 評価', 0.0029273708720882422),
 ('領域 探索', 0.002898447588298206),
 ('システム', 0.002897425905648082),
 ('ocds', 0.0028961280953063227),
 ('電気 刺激', 0.0028740095532096732),
 ('モルモット 蝸牛', 0.002857911433159567),
 ('問題点', 0.002848501574095293),
 ('cad', 0.0028417096855335466),
 ('体位 変換 手法', 0.0028259680922842595),
 ('yolo', 0.0028068607617351524),
 ('クロストーク', 0.0028012055030955715),
 ('超音波 画像', 0.0027953971487462473),
 ('乳房 mri 画像', 0